# Exp T13: Lifetime Measurement (Ordered → Collapse)

## 目的
Ordered状態からtarget λへ切り替えた時の崩壊までの寿命を測定。

## 実験設計
- **初期化**: λ=0.35で50 epoch学習（ordered状態確立）
- **Target λ**: 0.54（崩壊側、短寿命期待）
- **η**: 0.4
- **観測**: 崩壊イベント（error≥0.40が5 epoch継続）までの時間
- **打ち切り**: +300 epoch（censored）
- **Seeds**: 50

## 推定時間
崩壊率90%想定 → 平均15min/run → **~12h**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_T1_lifetime_ordered'
NOTEBOOK_ID = 'T13'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID} (O→C lifetime, λ=0.54, η=0.4)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

NOISE_RATE = 0.4
INIT_LAMBDA = 0.35
INIT_EPOCHS = 50
INIT_SUCCESS_THRESHOLD = 0.30
TARGET_LAMBDA = 0.54             # ★ 崩壊側（短寿命）
MAX_OBSERVE_EPOCHS = 300
COLLAPSE_THRESHOLD = 0.40
COLLAPSE_PERSIST = 5

N_SEEDS = 50

print(f'η = {NOISE_RATE}')
print(f'Init: λ={INIT_LAMBDA} for {INIT_EPOCHS} epochs')
print(f'Target: λ={TARGET_LAMBDA}, max {MAX_OBSERVE_EPOCHS} epochs')
print(f'Seeds: {N_SEEDS}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    cos_list = []
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        cos_list.append((gs_n @ gv_n).item())
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    if sched is not None:
        sched.step()
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv
    return np.mean(cos_list)

In [ ]:
def run_lifetime_experiment(seed, train_loader, test_loader, clean_labels, noisy_labels):
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    set_seed(seed)
    model = get_resnet18().to(device)
    opt = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    sched = optim.lr_scheduler.MultiStepLR(opt, [50, 75], 0.1)
    
    cached_gv_ref = {'step': 0, 'gv': None}
    init_trajectory = []
    
    print(f'    Phase 1: Init (λ={INIT_LAMBDA}, {INIT_EPOCHS}ep)...')
    for ep in range(INIT_EPOCHS):
        train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, INIT_LAMBDA, cached_gv_ref)
        if (ep + 1) % 10 == 0:
            err = 1 - evaluate(model, test_loader)
            init_trajectory.append({'epoch': ep + 1, 'error': err})
            print(f'      Ep {ep+1}: err={err:.4f}')
    
    init_final_error = 1 - evaluate(model, test_loader)
    
    if init_final_error > INIT_SUCCESS_THRESHOLD:
        print(f'    ⚠️ Init failed: {init_final_error:.4f}')
        return {
            'seed': seed, 'init_error': init_final_error, 'init_success': False,
            'lifetime': 0, 'censored': False, 'final_error': init_final_error,
            'init_trajectory': init_trajectory, 'observe_trajectory': []
        }
    
    print(f'    ✅ Init OK: {init_final_error:.4f}')
    print(f'    Phase 2: Observe (λ={TARGET_LAMBDA}, max {MAX_OBSERVE_EPOCHS}ep)...')
    
    observe_trajectory = [{'epoch': 0, 'error': init_final_error}]
    collapse_count = 0
    lifetime = None
    
    for ep in range(1, MAX_OBSERVE_EPOCHS + 1):
        train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, TARGET_LAMBDA, cached_gv_ref)
        err = 1 - evaluate(model, test_loader)
        observe_trajectory.append({'epoch': ep, 'error': err})
        
        if err >= COLLAPSE_THRESHOLD:
            collapse_count += 1
            if collapse_count >= COLLAPSE_PERSIST:
                lifetime = ep - COLLAPSE_PERSIST + 1
                print(f'      💀 Collapsed at ep {lifetime}')
                break
        else:
            collapse_count = 0
        
        if ep % 50 == 0:
            print(f'      Ep {ep}: err={err:.4f}')
    
    censored = (lifetime is None)
    if censored:
        lifetime = MAX_OBSERVE_EPOCHS
        print(f'      ⏱️ Censored at {MAX_OBSERVE_EPOCHS}ep')
    
    final_error = 1 - evaluate(model, test_loader)
    
    return {
        'seed': seed, 'init_error': init_final_error, 'init_success': True,
        'lifetime': lifetime, 'censored': censored, 'final_error': final_error,
        'init_trajectory': init_trajectory, 'observe_trajectory': observe_trajectory
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_seeds = set()

if os.path.exists(ckpt):
    results = json.load(open(ckpt))
    done_seeds = {r['seed'] for r in results}
    print(f'Loaded: {len(done_seeds)} done')

for seed in range(N_SEEDS):
    if seed in done_seeds:
        continue
    
    run_num = len(results) + 1
    print(f'\n{"="*50}')
    print(f'[{run_num}/{N_SEEDS}] Seed {seed} | λ={TARGET_LAMBDA}')
    print(f'{"="*50}')
    
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_lifetime_experiment(seed, train_loader, test_loader, clean_labels, noisy_labels)
    dt = time.time() - t0
    
    result['experiment_id'] = f'{NOTEBOOK_ID}-{seed:03d}'
    result['target_lambda'] = TARGET_LAMBDA
    result['eta'] = NOISE_RATE
    result['time'] = dt
    
    results.append(result)
    done_seeds.add(seed)
    
    if result['init_success']:
        status = '⏱️ CENSORED' if result['censored'] else f"💀 COLLAPSED@{result['lifetime']}"
    else:
        status = '❌ INIT FAILED'
    print(f'\n  {status} | Init:{result["init_error"]:.3f} → Final:{result["final_error"]:.3f} | {dt/60:.1f}min')
    
    json.dump(results, open(ckpt, 'w'), indent=2)
    
    successful = [r for r in results if r['init_success']]
    if successful:
        n_collapsed = sum(1 for r in successful if not r['censored'])
        print(f'  Progress: {n_collapsed}/{len(successful)} collapsed ({100*n_collapsed/len(successful):.0f}%)')
    
    remaining = N_SEEDS - run_num
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    torch.cuda.empty_cache()

print('\n' + '='*50 + f'\n{NOTEBOOK_ID} DONE\n' + '='*50)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2)
df = pd.DataFrame([{k: v for k, v in r.items() if 'trajectory' not in k} for r in results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY | λ={TARGET_LAMBDA}, η={NOISE_RATE}')
print('='*60)

successful = df[df['init_success'] == True]
if len(successful) > 0:
    n_collapsed = (successful['censored'] == False).sum()
    print(f'\n📊 Collapsed: {n_collapsed}/{len(successful)} ({100*n_collapsed/len(successful):.1f}%)')
    lifetimes = successful['lifetime'].values
    print(f'⏱️ Stats: mean={lifetimes.mean():.1f}, median={np.median(lifetimes):.1f}')

In [ ]:
successful = df[df['init_success'] == True]
if len(successful) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    lifetimes = successful['lifetime'].values
    
    ax = axes[0]
    times = np.arange(0, MAX_OBSERVE_EPOCHS + 1)
    survival = np.array([np.mean(lifetimes >= t) for t in times])
    ax.step(times, survival, where='post', linewidth=2, color='red')
    ax.fill_between(times, survival, step='post', alpha=0.2, color='red')
    ax.axhline(0.5, color='gray', linestyle='--', alpha=0.7)
    ax.set_xlabel('Time (epochs)')
    ax.set_ylabel('Survival Probability')
    ax.set_title(f'Survival Curve (λ={TARGET_LAMBDA})')
    ax.set_xlim(0, MAX_OBSERVE_EPOCHS)
    ax.set_ylim(0, 1.05)
    ax.grid(True, alpha=0.3)
    
    ax = axes[1]
    collapsed_lt = successful[successful['censored'] == False]['lifetime'].values
    if len(collapsed_lt) > 0:
        ax.hist(collapsed_lt, bins=20, color='red', alpha=0.7, edgecolor='black')
    ax.set_xlabel('Lifetime (epochs)')
    ax.set_ylabel('Count')
    ax.set_title('Lifetime Distribution')
    ax.grid(True, alpha=0.3)
    
    ax = axes[2]
    for r in results[:10]:
        if r['observe_trajectory'] and r['init_success']:
            eps = [t['epoch'] for t in r['observe_trajectory']]
            errs = [t['error'] for t in r['observe_trajectory']]
            ax.plot(eps, errs, alpha=0.5, color='red' if not r['censored'] else 'blue', linewidth=1)
    ax.axhline(COLLAPSE_THRESHOLD, color='orange', linestyle='--', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Test Error')
    ax.set_title('Sample Trajectories')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_survival.png', dpi=150)
    plt.show()